# NXP imx8qm x AWS NEO Object Detection Example

1. [Introduction](#Introduction)
2. [Compile model using NEO](#Compile-model-using-NEO)
3. [Inference on device](#Inference-on-device)

## Introduction

This notebook will demo how to compile pretrained Gluoncv ssd mobilenet model using AWS Neo for NXP imx8qm. First, we need download the pretained model, in this example we use `ssd_512_mobilenet1.0_voc`, then use Neo to cpmpile this model. At last, we will deploy compiled model to device and do inference using the Neo Deep Learning Runtime.

To get started, we need to set up the environment for AWS S3 permissions, configurations, and so on. Please refer to [Configuration](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html#configuration) for more information.

To use Boto 3, first import it and tell it what service we are going to use.

In [ ]:
import boto3
sm = boto3.client('sagemaker', region_name='us-west-2')
s3 = boto3.client('s3', region_name='us-west-2')

In [ ]:
account = boto3.client('sts').get_caller_identity().get('Account')
print(account)

### Get the pretrained model 

To avoid installing frameworks like mxnet, gluoncv on device to download latest pretained models, we store some pretrained models in a S3 bucket with public access. Download pretrained gluoncv ssd model from S3 bucket to device. 

In [ ]:
model_name = 'ssd_512_mobilenet1.0_voc'
model = model_name + '.tar.gz'
model_zoo = 'gluon_cv_object_detection'

In [ ]:
s3.download_file('neo-ai-dlr-test-artifacts', 
                 'neo-ai-notebook/{}/{}'.format(model_zoo, model), 
                 model)

### Upload model to S3 bucket

Create a S3 bucket `imx8qm-demo` to store pretrained model.

In [ ]:
bucket = 'imx8qm-demo'

In [ ]:
if boto3.resource('s3').Bucket(bucket) not in boto3.resource('s3').buckets.all():
    s3.create_bucket(
        Bucket=bucket,
        CreateBucketConfiguration={
            'LocationConstraint': 'us-west-2'
        }
    )
else:
    print('Bucket %s already exsits' %bucket)

Upload the pretained model to S3 bucket just created.

In [ ]:
s3.upload_file(model, bucket, model)

### Create IAM role
In order to use the sagemaker service and have access to S3 bucket, we need to create a IAM role.

In [ ]:
iam = boto3.client('iam')
role_name = 'imx8qm-demo-test-role'

In [ ]:
policy = {
    'Statement': [{
        'Action': 'sts:AssumeRole',
        'Effect': 'Allow',
        'Principal': {'Service': 'sagemaker.amazonaws.com'}},
        ],  
     'Version': '2012-10-17'}

In [ ]:
import json

roles = iam.list_roles()
role_arn = None
for role in roles['Roles']:
    if role['RoleName'] == role_name:
        role_arn = role['Arn']
        
if role_arn == None:
    new_role = iam.create_role(
        AssumeRolePolicyDocument=json.dumps(policy),
        Path='/',
        RoleName=role_name,
    )
    role_arn = new_role['Role']['Arn']

In [ ]:
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)

In [ ]:
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

## Compile model using NEO

In [ ]:
s3_output_location = 's3://{}/output'.format(bucket)
data_shape = '{"data":[1,3,512,512]}'
framework = 'mxnet'
target_device = 'imx8qm'

In [ ]:
import time
compilation_job_name = 'imx8qm-demo'+ str(time.time()).split('.')[0]
print('Compilation job for %s started' % compilation_job_name)

response = sm.create_compilation_job(
        CompilationJobName=compilation_job_name,
        RoleArn=role_arn,
        InputConfig={
            'S3Uri': 's3://{}/{}'.format(bucket, model),
            'DataInputConfig': data_shape,
            'Framework': framework.upper()
        },
        OutputConfig={
            'S3OutputLocation': s3_output_location,
            'TargetDevice': target_device 
        },
        StoppingCondition={
            'MaxRuntimeInSeconds': 900
        }
    )

print(response)

# Poll every 30 sec
while True:
    response = sm.describe_compilation_job(CompilationJobName=compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

## Inference on device

### Download compiled model from S3 to device

In [ ]:
object_path = 'output/{}-{}.tar.gz'.format(model_name, target_device)
neo_compiled_model = 'compiled-'+ model
s3.download_file(bucket, object_path, neo_compiled_model)

In [ ]:
%%bash 
mkdir compiled_model
tar -xf $neo_compiled_model -C ./compiled_model

### Use DLR to read compiled model

In [ ]:
from dlr import DLRModel
import numpy as np
import time

In [ ]:
# Load the model
model_path = "./compiled_model"

In [ ]:
device = 'cpu'
model = DLRModel(model_path, device)

### Download an image to prepare for predictions

In [ ]:
%%bash
wget -O test.jpg https://upload.wikimedia.org/wikipedia/commons/c/c6/Newark-broad-street.jpg

In [ ]:
file_name = "test.jpg"

# test image
from IPython.display import Image
Image(file_name)

### Image pre-process

In [ ]:
import PIL.Image
image = PIL.Image.open(file_name)

image = np.asarray(image.resize((512, 512)))

# Normalize
mean_vec = np.array([0.485, 0.456, 0.406])
stddev_vec = np.array([0.229, 0.224, 0.225])
image = (image/255- mean_vec)/stddev_vec

# Transpose
if len(image.shape) == 2:  # for greyscale image
    image = np.expand_dims(image, axis=2)
    
image = np.rollaxis(image, axis=2, start=0)[np.newaxis, :]

print(image.shape)

### Inference and prediction

In [ ]:
#flatten within a input array
input_data = {'data': image}

# dry run
for _ in range(5):
    model.run(input_data)

print('Testing inference...')
start_time = time.time()
detection = model.run(input_data) #need to be a list of input arrays matching input names
print('inference time is ' + str((time.time()-start_time)) + ' seconds')

In [ ]:
object_categories = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 
                     'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 
                     'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

obj = detection[0][0]
score = detection[1][0] 
bbox = detection[2][0]

# print the first 10 detetction
for i in range(10):
    print(object_categories[int(obj[i][0])] + ' with score ' + str(score[i][0]))